In [1]:
import io
import re
import csv
import time
import xlwt
import yaml
import math
import random
import numpy as np
import pandas as pd
from xlwt import Workbook
from datetime import datetime, date

In [2]:
start_time = time.time()
cityData = pd.read_csv("cityData.csv")
cityData.head()

,citylist,city,state_id,state_name,county_name,population,median age,family_size,income_individual_median,unemployment rate,...,Burglary,Larceny-theft,Motor vehicle theft,Arson2,income_household_median,six figure household percentage,home_value,2019 population growth rate,summer avg temp F,winter avg temp F
0,Abilene TX,Abilene,TX,Texas,Taylor,123420,31.6,3.19,25671,3.6,...,734.0,2561.0,233.0,3.0,50659,19.7,118441,0.53%,84.3,50.4
1,Akron OH,Akron,OH,Ohio,Suit,565208,36.9,3.05,23320,8.1,...,1740.0,4701.0,718.0,82.0,38739,11.8,81935,-0.15%,70.1,32.8
2,Albany GA,Albany,GA,Georgia,Dougherty,89217,34.3,3.22,20954,13.7,...,760.0,2666.0,278.0,26.0,36615,11.5,100932,-1.49%,80.8,53.7
3,Albany NY,Albany,NY,New York,Albany,586383,30.9,3.11,25436,7.0,...,432.0,2585.0,130.0,21.0,45825,20.2,178294,-0.57%,67.9,30.3
4,Albuquerque NM,Albuquerque,NM,New Mexico,Bernalillo,761195,36.6,3.23,29538,5.6,...,NaN,NaN,NaN,NaN,52911,22.7,198035,0.16%,75.4,38.6


In [3]:
# Dump Data to yaml 
content = pd.read_csv("OpeningParagraphTest.csv")
header = ["related_metric", "data_ranking","Python Copy"]
orderList = [""]*len(header)
dumpData = {}
    
for j in range(len(content)):
    ai_content = content.loc[j,header].values.flatten().tolist()
    for i in range(len(ai_content)):
        try: 
            if math.isnan(ai_content[i]):
                ai_content[i] = ""
        except:
            orderList[i] = ai_content[i]
        if ai_content[i] != "" and i != len(orderList) - 1:
            if i == 0:
                dumpData[orderList[i]] = {}
                key = orderList[i]
                continue
            dumpData[key].update({orderList[i]: ai_content[i + 1]})


with open(r'content.yml', 'w') as file:
    documents = yaml.dump(dumpData, file)



In [4]:
# Create the dictionary and create ranking columns
def meansort(attribute ,attributeList):
    columnMean = round(attributeList[attribute].mean(),2)
    x = attributeList.reset_index()[attributeList.columns].values.tolist()
    return sorted(x, key=lambda x: abs(x[1] - columnMean), reverse = True)


def createRankColumns(cityData):
    count = 0
    neutralListMap = {}
    mydict = {}

    
    with open('RankingConfigTest.csv', mode='r') as infile:
        reader = csv.reader(infile)
        next(reader)
        mydict = {rows[0].strip():[rows[1], rows[2], rows[3]] for rows in reader}
        infile.close()
    print(len(mydict))
    print(mydict)

    matrixList = cityData.iloc[:, 5:55]
    print("_+_+_+_+_+_ method called +_+_+_+_+_")
    for attribute in matrixList.columns:
    #     print(count, attribute)
        count+=1
        if (attribute in mydict.keys()) and (mydict[attribute.strip()][0] != "N/A"):
            cityData[attribute+"_rank"] = cityData[attribute].rank(ascending = mydict[attribute][1] == "FALSE", method="dense")
            if mydict[attribute][1] == "N/A":
#                 cityData[attribute+"_rank"] = cityData[attribute].rank(ascending = mydict[attribute][1] == "FALSE")
                neutralListMap[attribute] = meansort(attribute, cityData[["city", attribute]])
    return mydict, neutralListMap
# print(neutralListMap["avg. rainfall per year (mm)"])



In [5]:
# cityData[["city","avg. rainfall per year (mm)_rank"]]

In [6]:
# Given the attribute, buckets corresponding to the attribue and the city ranking corresponding to the
#  attribute identify the bucket the data falls in

def fetchkey(cityData, att, bucketList, rank, mydict):
    bucketList = list(bucketList)
    res = "".join([i for i in bucketList if "middle" in i])
    op = ""
    flag = True
    for i in range(len(bucketList)):
        print("Identify bucket for attribut = ", att, "Attribute Buckets = ", bucketList, "CityRank for the attribute = ",rank)
        if att == "air quality":
            return fetchAirQualityKey(rank)
        if not bucketList[i].endswith("%"):
            if int(rank) < 10:
                return bucketList[i]
#             else:
#                 return bucketList[i]
        else:
            if (res != "" and flag and mydict[att][1] != "N/A"):
                flag = False
                op = res
            if bucketList[i].startswith("top"):
                percentCheck = bucketList[i].replace("top", "").replace("%", "")
                columnRankCount = cityData[att].nunique()
                endRank = (int(percentCheck)/100)*columnRankCount
                print("Rank = ",rank, "Unique rank count = ", columnRankCount,"range", endRank,"precent", percentCheck)
                if rank <= endRank:
                    op = bucketList[i]
            elif(bucketList[i].startswith("bottom")):
                percentCheck = bucketList[i].replace("bottom", "").replace("%", "")
                columnRankCount = cityData[att].nunique()
                endRank = (int(percentCheck)/100)*columnRankCount
                print("rank", rank, "endRank = ", endRank, "endRank = ", endRank, "columnRankCount", columnRankCount, "difference check", columnRankCount - endRank)
                if (rank >= (columnRankCount - endRank)):
                    op = bucketList[i]
    return op
                
def fetchAirQualityKey(rank):
    if rank >= 1 and rank <=50:
        return "0-50"
    if rank >= 51 and rank <=100:
        return "51-100"
    if rank >= 101 and rank <=150:
        return "101-150"
    if rank >= 151 and rank <=200:
        return "151-200"
    if rank >= 201 and rank <=300:
        return "201-300"
    if rank >= 301 and rank <= 500:
        return "301-500"
    

In [7]:
def replace(city, stateId, att, sentence, rank):
#     sentence = sentence.replace("{city}", city)
#     sentence = sentence.replace("{city}'s", city+"'s'")
    print("$$$$$$$$$",cityData[att+"_rank"].nunique(), rank)
    sentence = sentence.replace("{rank}", numToOrdinal(rank))
    sentence = sentence.replace("{inverse_rank}", numToOrdinal(cityData[att+"_rank"].nunique() - rank))
    keywordList = re.findall("[{][\w.,&-/(/)/\s]{0,}[}]", sentence)
    for keyword in keywordList:
        dataToReplace = cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)][keyword[1:-1]].values[0]
        if not isinstance(dataToReplace, str):
            dataToReplace = numberFormat(dataToReplace)
        sentence = sentence.replace(keyword, dataToReplace)
    return sentence

def numberFormat(num):
    if not isinstance(num, float):
        num = str("{:,}".format(int(num)))
    else:
        num = ("{:,}".format(num)).rstrip("0").rstrip(".")
    return num

def numToOrdinal(value):
    if value % 100//10 != 1:
        if value % 10 == 1:
            ordval = u"%d%s" % (value, "st")
        elif value % 10 == 2:
            ordval = u"%d%s" % (value, "nd")
        elif value % 10 == 3:
            ordval = u"%d%s" % (value, "rd")
        else:
            ordval = u"%d%s" % (value, "th")
    else:
        ordval = u"%d%s" % (value, "th")

    return ordval



In [8]:
def fetchSentences(city, stateId, att, rank, key, descriptionData):
    print("key to check from the yaml for attribute = ", att," and key = ", key)
    sentenceList = descriptionData[att][key].split("|")
    sentence = random.choice(sentenceList)
    sentence = replace(city, stateId, att ,sentence, rank)  
    return sentence

In [9]:
def generateParagraph(city, stateId, metricRankList, cityParagraph, mydict):
    cityParagraph = ""
    bucketKeyList = []
    print("*********()()()()()()()*****", metricRankList)
    with open("content.yml") as file:
        descriptionData = yaml.full_load(file)
    for i in range(len(metricRankList)):
        if math.isnan(metricRankList[i][1]):
            continue
        key = fetchkey(cityData, metricRankList[i][0], descriptionData[metricRankList[i][0]].keys(), metricRankList[i][1], mydict)
        if key != "":
            res = fetchSentences(city, stateId, metricRankList[i][0], metricRankList[i][1], key, descriptionData)
            print("\n", "Random sentence that was choosen ", res, "\n")
            bucketKeyList.append(key)
#             cityParagraph += res.strip() + " "
            cityParagraph += res
            res =""
#                 res =""
    return cityParagraph, bucketKeyList

In [10]:
row = 1
wb = Workbook()
sheet1 = wb.add_sheet('Sheet 1')
mydict, neutralListMap = createRankColumns(cityData)
for cityIndex in range(len(cityData)):
    citylistData = (cityData.iloc[[cityIndex]]).values
    city = citylistData[0][1]
    stateId = citylistData[0][2]
    col = 0
    if city != "Burlington":
        continue
    posAttributeList = []
    negAttributeList = []
    neutralAttributeList = []
    fixedList = []
    counter = 0
    citycontent = cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)]
    rankColumns = cityData.iloc[:, 55:]
    rankData = citycontent[rankColumns.columns].values
    for i in rankColumns.columns:
        print(i)
        metric = i.replace("_rank", "")
        metrixData = dict(zip(cityData["city"], cityData[metric]))
        if metric not in mydict:
            counter += 1
            continue
        if pd.isna(metrixData[city]):
            counter += 1
            continue
        if metric == "2019 job opportunity":           
            fixedList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "TRUE":
            posAttributeList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "FALSE":
            negAttributeList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "N/A":
            neutralAttributeList.append((metric, neutralListMap[metric].index([city, metrixData[city]]) + 1))
        counter += 1
    print("\n ----------------------------------- Lists ----------------------------------------------- \n")
    print("City in consideration", city, citylistData[0][2])
    print("Fixed List", fixedList)
    print("Pos List", posAttributeList)
    print("Neg List", negAttributeList)
    print("Neu List", neutralAttributeList)
    print("\n ----------------------------------- Lists ----------------------------------------------- \n")
    fixedList.extend(sorted(posAttributeList, key=lambda x: x[1])[0:2])
    fixedList.extend(sorted(negAttributeList, key=lambda x: x[1])[0:1])
    fixedList.extend(sorted(neutralAttributeList, key=lambda x: x[1])[0:1])
    print("\n Final list of attributes \n", fixedList, "\n\n")
    fixedList[-1] = (fixedList[-1][0] , cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)][fixedList[-1][0]+"_rank"].values[0])
    print("\n Final list of attributes \n", fixedList, "\n\n")
    cityDescriptionParagraph, bucketKeyList = generateParagraph(city, citylistData[0][2] ,fixedList, "", mydict)
    print("################################################################## \n\n" ,row,city, cityDescriptionParagraph, "\n\n ##################################################################")
    sheet1.write(row, col, city)
    col += 1
    sheet1.write(row, col, cityDescriptionParagraph)
    col += 1
    sheet1.write(row, col, stateId)
    for i in range(len(bucketKeyList)):
        col += 1
        sheet1.write(row, col, bucketKeyList[i])
    row += 1
wb.save('generatedParagraphs.xls')
endTime = datetime.now()

print("--- %s Script Runtime  ---" % (time.time() - start_time))

# print(rankColumns.columns)
# print(rankData)
# category = []
# rankColumns = cityData.iloc[:, 51:]
# rankData = newYorkData[rankColumns.columns].values
# for i in rankColumns.columns:
#     category.append(mydict[i.replace("_rank", "")][2])
# rankData, rankColumns = zip(*sorted(zip(category, zip(rankData[0], rankColumns))))
# # rankData, rankColumns = zip(*sorted(zip(rankData[0], rankColumns)))
# # rankColumns[0] = m
# print(rankColumns)
# print(rankData)


25
{'air quality': ['yes', 'FALSE', 'B'], 'avg. rainfall per year (mm)': ['yes', 'N/A', 'C'], 'avg. snow per year (mm)': ['yes', 'N/A', 'C'], 'commute time (min)': ['yes', 'FALSE', 'A'], 'density': ['yes', 'N/A', 'B'], 'expend, per student': ['top', 'TRUE', 'C'], 'GDP': ['yes', 'TRUE', 'B'], 'Good&service index': ['yes', 'TRUE', 'A'], 'Grocery Index': ['yes', 'TRUE', 'A'], 'Healthcare index': ['yes', 'TRUE', 'A'], 'Housing Index': ['yes', 'TRUE', 'A'], '2019 job opportunity': ['yes', 'TRUE', 'A'], 'median age': ['bottom', 'N/A', 'C'], 'median home value': ['yes', 'FALSE', 'B'], 'median home value growth percentage': ['N/A', 'N/A', 'B'], 'median household income': ['N/A', 'TRUE', 'B'], 'population': ['yes', 'N/A', 'A'], 'population growth percentage': ['N/A', 'N/A', 'B'], 'poverty rate': ['yes', 'FALSE', 'B'], 'school_record': ['top', 'TRUE', 'C'], 'six figure household percentage': ['N/A', 'TRUE', 'C'], 'summer high': ['yes', 'N/A', 'C'], 'unemployment rate': ['bottom', 'FALSE', 'B'], 

Rank =  95.0 Unique rank count =  193 range 48.25 precent 25
Identify bucket for attribut =  unemployment rate Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  16.0
rank 16.0 endRank =  19.5 endRank =  19.5 columnRankCount 78 difference check 58.5
Identify bucket for attribut =  unemployment rate Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  16.0
Identify bucket for attribut =  unemployment rate Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  16.0
Rank =  16.0 Unique rank count =  78 range 19.5 precent 25
key to check from the yaml for attribute =  unemployment rate  and key =  top25%
$$$$$$$$$ 78 16.0

 Random sentence that was choosen  Burlington's high unemployment rate of 4.3, which ranks the 16th highest on our city list, means potentially more job applicants and possibly higher competition for available jobs.  

Identify bucket for attribut =  winter low Attrib

In [11]:
# count = 0
# start = 0
# end = len(rankColumns) - 1
# bucketList = []
# cityParagraph = ""
# res = ""
# with open("content.yml") as file:
#     descriptionData = yaml.full_load(file)

# while(start < len(rankColumns) and start < end):
# #     if count == 4:
# #         print("ran break successfully")
# #         break
# #     if count < 3:
# # #         print(rankColumns)
# #         positiveAttribute = rankColumns[start][1].replace("_rank", "")
#         if (mydict[positiveAttribute][0] != "N/A") and (mydict[positiveAttribute][0] == "top" or mydict[positiveAttribute][0] == "yes"):  
#             print(type(descriptionData[positiveAttribute].keys()))
#             key = fetchkey(rankColumns[start], descriptionData[positiveAttribute].keys(), rankColumns[start][0])
#             print("returned key from the fetch function is =======", key)
#             if key != None:
#                 count += 1
#                 res = fetchSentences(positiveAttribute, key, descriptionData)
#         start += 1
        
#     else:
#         negativeAttribute = rankColumns[end][1].replace("_rank", "")
#         if mydict[positiveAttribute][0] != "N/A" and mydict[negativeAttribute][0] != "top":
#             print("*****",rankColumns[end])
#             print("****",descriptionData[negativeAttribute].keys())
#             print("***",rankData[end])
#             key = fetchkey(rankColumns[end], descriptionData[negativeAttribute].keys(), rankColumns[start][0])
#             if key != None:
#                 count += 1
#                 res = fetchSentences(negativeAttribute, key, descriptionData)
#         end -= 1
#     cityParagraph += res
#     res =""        


# print("###############################################################")
# print(cityParagraph)
# print("###############################################################")

In [12]:
# cityData["avg. snow per year_rank"].nunique()

In [13]:
# New York
# City_name's average commute time is pretty long: about 23.1 minutes. This is the 23rd longest commute time on our city list.City_name is a small city. It has only xpopulation people living in it, ranking it as the xrank smallest population in our city list. There are 552 schools per 100,000 people, making City_name a place with good access to education. This ranks the city 58th on our city list. City_name's winter is very cold. The average temperature in winter is just XX°F, ranking this as the 42nd coldest winter city. 

In [14]:
# ('commute time_rank', 'population_rank', 'schools per 100,000 people_rank', 'median home value_rank', 'expend, per student_rank', 'avg. rainfall per year_rank', 'air quality_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'median age_rank', 'poverty rate_rank', 'Healthcare index_rank', 'Utilities index_rank', 'Good&service index_rank', 'GDP_rank', 'avg. snow per year_rank', 'Housing Index_rank', 'median household income growth_rank')
# (1.0, 1.0, 1.0, 3.0, 7.0, 12.0, 14.0, 43.0, 43.0, 47.0, 82.0, 108.0, 113.0, 115.0, 124.0, 124.0, 145.0, 159.0)


# ('Utilities index_rank', 'GDP_rank', 'Good&service index_rank', 'Healthcare index_rank', 'avg. snow per year_rank', 'schools per 100,000 people_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'Housing Index_rank', 'median household income growth_rank', 'poverty rate_rank', 'median age_rank', 'median home value_rank', 'commute time_rank', 'avg. rainfall per year_rank', 'population_rank', 'expend, per student_rank', 'air quality_rank')
# (3.0, 13.0, 24.0, 27.0, 37.0, 64.0, 69.0, 69.0, 83.0, 92.0, 93.0, 94.0, 109.0, 115.0, 144.0, 152.0, 196.0, nan)

# ('air quality_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'avg. snow per year_rank', 'avg. rainfall per year_rank', 'schools per 100,000 people_rank', 'poverty rate_rank', 'median age_rank', 'expend, per student_rank', 'commute time_rank', 'population_rank', 'Healthcare index_rank', 'Good&service index_rank', 'median household income growth_rank', 'Housing Index_rank', 'Utilities index_rank', 'GDP_rank', 'median home value_rank')
# (19.0, 26.0, 26.0, 27.0, 38.0, 43.0, 44.0, 45.0, 55.0, 60.0, 65.0, 66.0, 75.0, 75.0, 82.0, 83.0, 121.0, 135.0)

# ('Job opportunity_rank', 'unemployment rate_rank', 'poverty rate_rank', 'air quality_rank', 'expend, per student_rank', 'Housing Index_rank', 'median age_rank', 'Good&service index_rank', 'Utilities index_rank', 'schools per 100,000 people_rank', 'avg. rainfall per year_rank', 'median household income growth_rank', 'Healthcare index_rank', 'commute time_rank', 'avg. snow per year_rank', 'median home value_rank', 'GDP_rank', 'population_rank')
# (4.0, 4.0, 14.0, 24.0, 43.0, 51.0, 70.0, 72.0, 76.0, 79.0, 85.0, 89.0, 92.0, 95.0, 122.0, 129.0, 161.0, 174.0)


In [15]:
# Test for preprocessing

# cityData.head()
# cityData["avg. rainfall per year"]
# cityData["avg. snow per year"]
# empDfObj = pd.DataFrame(cityData, columns=["avg. rainfall per year", "avg. snow per year", "city"])
# print(empDfObj.dtypes)

In [16]:
# Comparator test
# d = {'col1': [1, 2, 3,4,5,21], 'col2': [3, 4,7,8,9,1]}
# df = pd.DataFrame(data=d)
# m = df["col1"].mean()
# xy = df.reset_index()[df.columns].values.tolist()
# df

In [17]:
# print(xy)
# print(m)

In [18]:
# sorted(xy, key=lambda x: abs(x[0] - m), reverse = True)
# xy

In [19]:
# t = map({xy.split(0), xy.split(0)},xy)
# t

In [20]:
# Sort by key
# import statistics

# import operator
# x = {1: 21, 3: 1, 4: 2, 2: 3, 0: 4, 11: 5}
# print(x.items())
# m1 = statistics.mean(x.values())
# print(m1)
# sorted_x = sorted(x.items(), key=lambda x: abs(x[1] - m1), reverse=True)
# print(sorted_x)

In [21]:
# a = [10.0, 9.8, 8.0, 7.8, 7.7, 7.0, 6.0, 5.0, 4.0, 2.0, 2.0]
# rank = lambda arr: list(map(lambda i: sorted(arr).index(i)+1, arr))
# rank(a)

In [22]:
# x = ['bottom25%', 'middle50%', 'top25%']
# res = "".join([i for i in x if "middle" in i])
# res = any("middle" in string for string in x)
# op =""
# flag=True

# for i in range(0,3):
#     if (res and flag):
#         flag = False
#         op = x[i]
#         print("Middle executed")
#     if x[i].startswith("top"):
#         print("Tiop executed")
#     elif x[i].startswith("bottom"):
#         print("Bottom executed executed")
# print(res)

In [23]:
# 129 >= 173 - 43.25


In [24]:
# def abc():
#     return "a", [1,2,3]

# x, y = abc()
# print(x)
# print(y)

In [25]:
# "m name is {city}'s".replace("{city}'s", "New York's")

In [26]:
# import re
# s = "Summers in {city} are temperate, with an average temperature of {summer_avg_temp}°F and a high of {summer high}°F. This ranks it midway at {rank} on our summer city list. |"
# x = re.findall("[{][\w.&,-/(/)/\s]{0,}[}]", s)
# print(x)
# for i in x:
#     print(i[1:-1])
#     s = s.replace(i, str(cityData[np.logical_and(cityData["city"] == "Abilene", cityData["state_id"] == "TX")][i[1:-1]].values[0]))
# #     print(i[1:-1])
# #     p = s.find(i)
# #     print(p)
# #     s = s.replace(i, str(100))
# print(s)

In [27]:
# x = cityData[cityData["city"] == "New York"]["population"]
# print(int(x))

In [28]:
# cityData[cityData["city"] == "New York"]["population"]

In [29]:
# (cityData[np.logical_and(cityData["city"] == "Albany", cityData["state_id"] == "GA")]["2019 job opportunity"].values)[0]

In [30]:
# for cityIndex in range(len(cityData)):
#     city = (cityData.iloc[[cityIndex]]).values
#     print(city[0][1])
#     break

In [31]:
# print(datetime.now())

In [32]:
# citylistData = (cityData.iloc[[1]]).values
# citylistData

In [33]:
# cityData[["city","population_rank"]]

In [34]:
# cityData[np.logical_and(cityData["city"] == "Albuquerque", cityData["state_id"] == "NM")]["avg. rainfall per year (mm)_rank"].values[0]

In [35]:
# def rescue_code(function):
#     import inspect
#     get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))

In [36]:
# rescue_code(fetchAirQualityKey)

In [37]:
# def test_meansort():
#     x = main.meansort(main.cityData["winter low"], df[["Name", "winter low"]])
#     print(x)
# test_meansort()